In [31]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap
from matplotlib import pyplot as plt
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [2]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [4]:
plt.rcParams.update({'figure.figsize': (12.0, 8.0)})
plt.rcParams.update({'font.size': 14})

In [5]:
file_path = "20_Group_4_Final_Project/Resources/modern_season_stats_clean.csv"
seasons_df = pd.read_csv(file_path)
seasons_df

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,id,AllStar
0,1980,Kareem Abdul-Jabbar,C,32,LAL,82,0,3143,25.3,0.639,...,696,886,371,81,280,297,216,2034,0,1
1,1980,Tom Abernethy,PF,25,GSW,67,0,1222,11.0,0.511,...,129,191,87,35,12,39,118,362,1,0
2,1980,Alvan Adams,C,25,PHO,75,0,2168,19.2,0.571,...,451,609,322,108,55,218,237,1118,2,0
3,1980,Tiny Archibald,PG,31,BOS,80,80,2864,15.3,0.574,...,138,197,671,106,10,242,218,1131,3,1
4,1980,Dennis Awtrey,C,31,CHI,26,0,560,7.4,0.524,...,86,115,40,12,15,27,66,86,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,2017,Cody Zeller,PF,24,CHO,62,58,1725,16.7,0.604,...,270,405,99,62,58,65,189,639,18922,0
18923,2017,Tyler Zeller,C,27,BOS,51,5,525,13.0,0.508,...,81,124,42,7,21,20,61,178,18923,0
18924,2017,Stephen Zimmerman,C,20,ORL,19,0,108,7.3,0.346,...,24,35,4,2,5,3,17,23,18924,0
18925,2017,Paul Zipser,SF,22,CHI,44,18,843,6.9,0.503,...,110,125,36,15,16,40,78,240,18925,0


In [8]:
X = seasons_df.copy().drop("AllStar", axis=1)
X

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,id
0,1980,Kareem Abdul-Jabbar,C,32,LAL,82,0,3143,25.3,0.639,...,190,696,886,371,81,280,297,216,2034,0
1,1980,Tom Abernethy,PF,25,GSW,67,0,1222,11.0,0.511,...,62,129,191,87,35,12,39,118,362,1
2,1980,Alvan Adams,C,25,PHO,75,0,2168,19.2,0.571,...,158,451,609,322,108,55,218,237,1118,2
3,1980,Tiny Archibald,PG,31,BOS,80,80,2864,15.3,0.574,...,59,138,197,671,106,10,242,218,1131,3
4,1980,Dennis Awtrey,C,31,CHI,26,0,560,7.4,0.524,...,29,86,115,40,12,15,27,66,86,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,2017,Cody Zeller,PF,24,CHO,62,58,1725,16.7,0.604,...,135,270,405,99,62,58,65,189,639,18922
18923,2017,Tyler Zeller,C,27,BOS,51,5,525,13.0,0.508,...,43,81,124,42,7,21,20,61,178,18923
18924,2017,Stephen Zimmerman,C,20,ORL,19,0,108,7.3,0.346,...,11,24,35,4,2,5,3,17,23,18924
18925,2017,Paul Zipser,SF,22,CHI,44,18,843,6.9,0.503,...,15,110,125,36,15,16,40,78,240,18925


In [9]:
y = seasons_df["AllStar"]
y

0        1
1        0
2        0
3        1
4        0
        ..
18922    0
18923    0
18924    0
18925    0
18926    0
Name: AllStar, Length: 18927, dtype: int64

In [11]:
X.columns

Index(['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%',
       'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PF', 'PTS', 'id'],
      dtype='object')

In [14]:
X = X.copy().drop(['Player','Year','Tm','id'], axis=1)
X

#df.drop(['C', 'D'], axis = 1)

,Pos,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,C,32,82,0,3143,25.3,0.639,0.001,0.344,7.2,...,0.765,190,696,886,371,81,280,297,216,2034
1,PF,25,67,0,1222,11.0,0.511,0.003,0.258,5.4,...,0.683,62,129,191,87,35,12,39,118,362
2,C,25,75,0,2168,19.2,0.571,0.002,0.270,8.2,...,0.797,158,451,609,322,108,55,218,237,1118
3,PG,31,80,80,2864,15.3,0.574,0.023,0.548,2.3,...,0.830,59,138,197,671,106,10,242,218,1131
4,C,31,26,0,560,7.4,0.524,0.000,0.833,6.0,...,0.640,29,86,115,40,12,15,27,66,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,PF,24,62,58,1725,16.7,0.604,0.002,0.442,8.6,...,0.679,135,270,405,99,62,58,65,189,639
18923,C,27,51,5,525,13.0,0.508,0.006,0.247,9.2,...,0.564,43,81,124,42,7,21,20,61,178
18924,C,20,19,0,108,7.3,0.346,0.000,0.161,10.8,...,0.600,11,24,35,4,2,5,3,17,23
18925,SF,22,44,18,843,6.9,0.503,0.448,0.181,1.9,...,0.775,15,110,125,36,15,16,40,78,240


In [15]:
X = pd.get_dummies(X, columns=["Pos"])
X

#X = pd.get_dummies(df_cluster, columns=["Algorithm","ProofType"])
#X


,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,...,Pos_PG,Pos_PG-SF,Pos_PG-SG,Pos_SF,Pos_SF-PF,Pos_SF-SG,Pos_SG,Pos_SG-PF,Pos_SG-PG,Pos_SG-SF
0,32,82,0,3143,25.3,0.639,0.001,0.344,7.2,22.2,...,0,0,0,0,0,0,0,0,0,0
1,25,67,0,1222,11.0,0.511,0.003,0.258,5.4,12.0,...,0,0,0,0,0,0,0,0,0,0
2,25,75,0,2168,19.2,0.571,0.002,0.270,8.2,22.4,...,0,0,0,0,0,0,0,0,0,0
3,31,80,80,2864,15.3,0.574,0.023,0.548,2.3,5.3,...,1,0,0,0,0,0,0,0,0,0
4,31,26,0,560,7.4,0.524,0.000,0.833,6.0,16.9,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,24,62,58,1725,16.7,0.604,0.002,0.442,8.6,17.3,...,0,0,0,0,0,0,0,0,0,0
18923,27,51,5,525,13.0,0.508,0.006,0.247,9.2,17.0,...,0,0,0,0,0,0,0,0,0,0
18924,20,19,0,108,7.3,0.346,0.000,0.161,10.8,24.9,...,0,0,0,0,0,0,0,0,0,0
18925,22,44,18,843,6.9,0.503,0.448,0.181,1.9,14.2,...,0,0,0,1,0,0,0,0,0,0


In [16]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [22]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [33]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [34]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [35]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,4454,42
Actual 1,97,139


In [36]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [37]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4454,42
Actual 1,97,139


Accuracy Score : 0.970625528317836
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4496
           1       0.77      0.59      0.67       236

    accuracy                           0.97      4732
   macro avg       0.87      0.79      0.83      4732
weighted avg       0.97      0.97      0.97      4732



In [38]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.01014877, 0.01318667, 0.0146822 , 0.01789202, 0.05282184,
       0.01028454, 0.00786684, 0.01008125, 0.00878315, 0.01039572,
       0.01054202, 0.01326512, 0.00849773, 0.00807227, 0.00889342,
       0.02684867, 0.03614405, 0.02268983, 0.08942588, 0.03186901,
       0.02209902, 0.00982985, 0.03315485, 0.06228896, 0.03850851,
       0.03442714, 0.0086784 , 0.00764966, 0.00776228, 0.00711207,
       0.03323911, 0.03399884, 0.00882359, 0.00942665, 0.04490086,
       0.03808813, 0.01018705, 0.01014747, 0.01417676, 0.01326981,
       0.01292992, 0.01111359, 0.00996923, 0.02684018, 0.01305958,
       0.06109396, 0.00111796, 0.        , 0.        , 0.00099754,
       0.        , 0.        , 0.00044658, 0.        , 0.00013449,
       0.00098824, 0.        , 0.        , 0.00093554, 0.        ,
       0.00021318, 0.        ])

In [39]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.0894258814222068, 'WS'),
 (0.06228895906241635, 'VORP'),
 (0.06109395652151225, 'PTS'),
 (0.052821838431123695, 'PER'),
 (0.04490086288043776, 'FT'),
 (0.03850850740385605, 'FG'),
 (0.0380881344587839, 'FTA'),
 (0.03614405229286872, 'OWS'),
 (0.03442714437456703, 'FGA'),
 (0.03399884453900418, '2PA'),
 (0.03323911172071689, '2P'),
 (0.03315485069783894, 'BPM'),
 (0.03186900643777364, 'WS/48'),
 (0.02684867299779661, 'USG%'),
 (0.026840175756215764, 'TOV'),
 (0.022689827677208962, 'DWS'),
 (0.022099017481279314, 'OBPM'),
 (0.01789201770930082, 'MP'),
 (0.014682202718544906, 'GS'),
 (0.014176763480068277, 'DRB'),
 (0.013269808704863894, 'TRB'),
 (0.013265118183002458, 'AST%'),
 (0.013186667110275456, 'G'),
 (0.013059579151131484, 'PF'),
 (0.012929916235793116, 'AST'),
 (0.011113590160768645, 'STL'),
 (0.010542023892248302, 'TRB%'),
 (0.010395721815139345, 'DRB%'),
 (0.010284543203081585, 'TS%'),
 (0.010187051107272364, 'FT%'),
 (0.010148773646130307, 'Age'),
 (0.010147473555611018, '

In [ ]:
#X = seasons_df.copy().drop()
#y = boston.target
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12)

In [6]:
#df_crypto_clean.drop("IsTrading", axis=1).copy()